# Gallery Example: M/Erl/k Queue

This example demonstrates an M/Erlang/k queueing system:
- **Arrivals**: Exponential (Poisson process)
- **Service**: Erlang distribution (low variability)
- **Servers**: k servers (default k=2)
- **Capacity**: Infinite
- **Scheduling**: FCFS

The Erlang service with multiple servers provides regular, predictable processing.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_merlk(k=2): Network {
    """Create M/Erl/k queueing model"""
    val model = Network(f"M/Erl/{k}")
// Block 1: nodes
    val source = Source(model, "mySource")
    val queue = Queue(model, "myQueue", SchedStrategy.FCFS)
    val sink = Sink(model, "mySink")
// Block 2: classes
    val oclass = OpenClass(model, "myClass")
    source.setArrival(oclass, Exp(1))
// Erlang-2 service with mean=0.5
    queue.setService(oclass, Erlang.fit_mean_and_order(0.5, 2))
    queue.setNumberOfServers(k)
// Block 3: topology
    model.link(Network.serialRouting(source, queue, sink))
    
    return model
// Create the model with k=2 servers
val model = gallery_merlk(k=2)

## About M/Erl/k Queues

For M/Erl/k with k=2 servers and Erlang-2 service:
- **Arrival rate**: λ = 1
- **Service mean**: 0.5 (rate μ = 2 per server)
- **Service SCV**: 1/2 = 0.5 (Erlang order = 2)
- **Total capacity**: k × μ = 4
- **Utilization**: ρ = λ / (k × μ) = 1/4 = 0.25

The low service variability (Erlang) combined with multiple servers provides excellent performance.

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// CTMC Solver
val solver_ctmc = CTMC(model, "cutoff", 15)
val avg_table_ctmc = solver_ctmc.avgTable
println("\nCTMC Solver:")
println(avg_table_ctmc)
// Fluid Solver
val solver_fluid = FLD(model)
val avg_table_fluid = solver_fluid.avgTable
println("\nFluid Solver:")
println(avg_table_fluid)